# Часть 3: Использование готовых MCP-серверов

В этом примере мы используем готовый сервер `mcp-server-fetch` из официального репозитория Model Context Protocol.

Вместо написания собственного кода для работы с веб-страницами, мы просто устанавливаем пакет и подключаем его.

## Шаг 1: Импорты и настройка

In [6]:
import asyncio
import os

from dotenv import load_dotenv
from fastmcp import Client
from fastmcp.client.transports import StdioTransport
from langchain.agents import create_agent
from langchain_mcp_adapters.tools import load_mcp_tools
from langchain_openai import ChatOpenAI

import warnings
warnings.filterwarnings('ignore')

load_dotenv()

BASE_MODEL = os.getenv("BASE_MODEL") or "qwen/qwen3-235b-a22b-2507"

## Шаг 2: О сервере mcp-server-fetch

Этот сервер из [официального репозитория MCP](https://github.com/modelcontextprotocol/servers/tree/main/src/fetch) предоставляет:

- Загрузку веб-страниц через инструмент `fetch`
- Конвертацию HTML в markdown для удобной обработки моделями
- Параметры для контроля размера и чтения контента по частям

Установка: `uvx mcp-server-fetch` (или через npm/pip)

## Шаг 3: Подключение готового сервера

Создаем транспорт, который запускает сервер через `uvx`.
Процесс подключения идентичен собственным серверам.

In [7]:
async def main():
    transport = StdioTransport(command="uvx", args=["mcp-server-fetch"])

    async with Client(transport) as client:
        print("Available tools:")
        tools_list = await client.list_tools()
        for tool in tools_list:
            print(f"  - {tool.name}")

        tools = await load_mcp_tools(client.session)

        llm = ChatOpenAI(model=BASE_MODEL, temperature=0)

        agent = create_agent(llm, tools)

        response = await agent.ainvoke(
            {
                "messages": [
                    (
                        "user",
                        "Fetch the content from https://en.wikipedia.org/wiki/Lorem_ipsum and summarize key points. Use signle step of given tool and return response.",
                    )
                ]
            }
        )

        print("\nAgent response:")
        for message in response["messages"]:
            print(f"\n{message.content}")

## Шаг 4: Запуск

In [8]:
import nest_asyncio
nest_asyncio.apply()

In [9]:
asyncio.run(main())

Available tools:
  - fetch

Agent response:

Fetch the content from https://en.wikipedia.org/wiki/Lorem_ipsum and summarize key points. Use signle step of given tool and return response.



Contents of https://en.wikipedia.org/wiki/Lorem_ipsum:
From Wikipedia, the free encyclopedia

"Ipsum" redirects here. For the car, see [Toyota Ipsum](/wiki/Toyota_Ipsum "Toyota Ipsum").

[![An example of the Lorem ipsum placeholder text on a green and white webpage.](//upload.wikimedia.org/wikipedia/commons/thumb/8/86/Lorem_ipsum_design.svg/330px-Lorem_ipsum_design.svg.png)](/wiki/File:Lorem_ipsum_design.svg)

Using *Lorem ipsum* to focus attention on graphic elements in a webpage design proposal

[![](//upload.wikimedia.org/wikipedia/commons/thumb/4/42/Letraset_Lorem_Ipsum.jpg/250px-Letraset_Lorem_Ipsum.jpg)](/wiki/File:Letraset_Lorem_Ipsum.jpg)

An example of the *Lorem ipsum* placeholder text on a Letraset sample sheet. Date unknown, possibly 1970s.

***Lorem ipsum*** ( [*LOR-əm IP-səm*](/wiki/He

## Экосистема MCP и открытость

Hепозитории с готовыми серверами:

- [Docker Hub MCP](https://hub.docker.com/mcp/explore)
- [Glama MCP Servers](https://glama.ai/mcp/servers)
- Научные инструменты: [academia_mcp](https://github.com/IlyaGusev/academia_mcp)
- МСП для работы с автокадом и создания чертежей: https://github.com/puran-water/autocad-mcp

Вы можете:
- Использовать готовые решения других разработчиков
- Публиковать свои серверы для сообщества
- Контрибутить в существующие проекты